In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from imutils import paths
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout,Flatten,Dense,Input
from keras import Sequential
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.constants import lb
import numpy as np
import cv2 as cv
import imutils
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours

In [2]:
def midpoint(ptA, ptB):
	return (ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5

In [3]:
def matchTemplatePrivate(img):
    METHOD = cv.TM_CCOEFF

    # lê novamente a imagem para evitar dados quebrados
    edged_img = cv.adaptiveThreshold(img, 255,
                                     cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 21, 10)

    img2 = img.copy()

    # carrega template para joelho esquerdo e direito
    template_l = cv.imread("../templates/template_L.png", 0)
    template_r = cv.imread("../templates/template_R.png", 0)

    # encontra contornos
    edged_template_l = cv.adaptiveThreshold(template_r, 255,
                                            cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 21, 10)

    edged_template_r = cv.adaptiveThreshold(template_l, 255,
                                            cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 21, 10)

    w_l, h_l = template_l.shape[::-1]
    w_r, h_r = template_l.shape[::-1]

    # aplica o math template em ambas as imagens de template
    res_l = cv.matchTemplate(edged_img, edged_template_l, METHOD)
    res_r = cv.matchTemplate(edged_img, edged_template_r, METHOD)

    min_val_l, max_val_l, min_loc_l, max_loc_l = cv.minMaxLoc(res_l)
    min_val_r, max_val_r, min_loc_r, max_loc_r = cv.minMaxLoc(res_r)

    # define qual imagem deu melhor match
    if max_val_r > max_val_l:
        top_left = max_loc_r
        bottom_right = (top_left[0] + w_r, top_left[1] + h_r)
    else:
        top_left = max_loc_l
        bottom_right = (top_left[0] + w_l, top_left[1] + h_l)

    return top_left, bottom_right

In [4]:
def processImage(image):
    image = img_to_array(image, dtype='uint8')
    print(type(image))

    x, y = matchTemplatePrivate(image)
    print(x,y)
    image = image[x[1]:y[1],x[0]:y[0]]
    image = array_to_img(image)
    image = image.resize((224,224))
    #image.show()

    image = img_to_array(image, dtype='uint8')

    return image#cv.cvtColor(image,cv.COLOR_GRAY2RGB)

In [32]:
image_1 =cv.imread("C:\\Users\\T-GAMER\\Downloads\\KneeXrayData\\KneeXrayData\\ClsKLData\\kneeKL224\\train_preprocessed\\0\\9898582R.png", 0)
image_2 =cv.imread("C:\\Users\\T-GAMER\\Downloads\\KneeXrayData\\KneeXrayData\\ClsKLData\\kneeKL224\\train_preprocessed\\4\\9025994L.png", 0)

In [ ]:
print(type(image_2))


In [28]:
cv.imshow("image1", image_1)
cv.imshow("image2", image_2)
cv.waitKey(0)
cv.destroyAllWindows()

In [33]:
image_1 = processImage(image_1)
image_2 = processImage(image_2)


<class 'numpy.ndarray'>
(7, 79) (215, 164)
<class 'numpy.ndarray'>
(6, 77) (214, 162)


In [34]:
gray = cv.GaussianBlur(image_1, (7, 7), 0)
# perform edge detection, then perform a dilation + erosion to
# close gaps in between object edges
edged = cv.Canny(gray, 50, 100)
edged = cv.dilate(edged, None, iterations=1)
edged = cv.erode(edged, None, iterations=1)
# find contours in the edge map
cnts = cv.findContours(edged.copy(), cv.RETR_EXTERNAL,
	cv.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
(cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")
colors = ((0, 0, 255), (240, 0, 159), (0, 165, 255), (255, 255, 0),
	(255, 0, 255))
refObj = None

In [35]:
import math
menorDistancia = math.inf
distancias = 0
cont = 0

In [36]:
# loop over the contours individually
for c in cnts:
	# if the contour is not sufficiently large, ignore it
	if cv.contourArea(c) < 100:
		continue
	# compute the rotated bounding box of the contour
	box = cv.minAreaRect(c)
	box = cv.cv.BoxPoints(box) if imutils.is_cv2() else cv.boxPoints(box)
	box = np.array(box, dtype="int")
	# order the points in the contour such that they appear
	# in top-left, top-right, bottom-right, and bottom-left
	# order, then draw the outline of the rotated bounding
	# box
	box = perspective.order_points(box)
	# compute the center of the bounding box
	cX = np.average(box[:, 0])
	cY = np.average(box[:, 1])
    # if this is the first contour we are examining (i.e.,
	# the left-most contour), we presume this is the
	# reference object
	if refObj is None:
		# unpack the ordered bounding box, then compute the
		# midpoint between the top-left and top-right points,
		# followed by the midpoint between the top-right and
		# bottom-right
		(tl, tr, br, bl) = box
		(tlblX, tlblY) = midpoint(tl, bl)
		(trbrX, trbrY) = midpoint(tr, br)
		# compute the Euclidean distance between the midpoints,
		# then construct the reference object
		D = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
		refObj = (box, (cX, cY), D / 224)
		continue
    # draw the contours on the image
	orig = image_1.copy()
	cv.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
	cv.drawContours(orig, [refObj[0].astype("int")], -1, (0, 255, 0), 2)
	# stack the reference coordinates and the object coordinates
	# to include the object center
	refCoords = np.vstack([refObj[0], refObj[1]])
	objCoords = np.vstack([box, (cX, cY)])
    # loop over the original points
	for ((xA, yA), (xB, yB), color) in zip(refCoords, objCoords, colors):
		# draw circles corresponding to the current points and
		# connect them with a line
		cv.circle(orig, (int(xA), int(yA)), 5, color, -1)
		cv.circle(orig, (int(xB), int(yB)), 5, color, -1)
		cv.line(orig, (int(xA), int(yA)), (int(xB), int(yB)),
			color, 2)
		# compute the Euclidean distance between the coordinates,
		# and then convert the distance in pixels to distance in
		# units
		D = dist.euclidean((xA, yA), (xB, yB)) / refObj[2]

		distancias += D
		cont+=1
		if D< menorDistancia:
			menorDistancia = D

		(mX, mY) = midpoint((xA, yA), (xB, yB))
		cv.putText(orig, "{:.1f}in".format(D), (int(mX), int(mY - 10)),
			cv.FONT_HERSHEY_SIMPLEX, 0.55, color, 2)
	    # show the output image
		#cv.imshow("Image", orig)
		#cv.waitKey(0)
cv.waitKey(0)
cv.destroyAllWindows()

In [37]:
print(distancias/cont)
print(menorDistancia)

143.4461532652545
68.09589772291052
